In [ ]:
!pip install torch transformers openai  huggingface_hub safetensors accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.

In [ ]:
import os
import numpy as np
import openai
import json
import pandas as pd
import torch
import re
from sklearn.model_selection import train_test_split
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from huggingface_hub import login
hf_token = "input hugging face token"
login(hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
# !nvidia-smi

In [ ]:
csv_path = "./dataset/SPOTIFY_REVIEWS.csv"
df_new = pd.read_csv(csv_path)
df_new = df_new.fillna("NaN")
df_new = df_new.sample(n=1000, random_state=42) # Just take 1000 data for fine-tuning
len(df_new)

1000

In [ ]:
train_df, valid_df = train_test_split(df_new, test_size=0.3, random_state=42)
print(len(train_df))
print(len(valid_df))

700
300


In [ ]:
#EXAMPLE

In [ ]:
serialkey = "input-your-key"
openai.api_key = serialkey
MODEL = "gpt-4o-mini"

In [ ]:
def generate_paraphrase(review_text):
    """Generate a paraphrased version of the review text using GPT-4 API."""
    try:
        # Create the completion request
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "user",
                    "content": f"You are a Spotify App review assistant. Your job is to paraphrase the following review as the third person (using wording such as 'Our user stated that'): '{review_text}'"
                }
            ],
            max_tokens=100,  # Adjust based on desired output length
            temperature=0.7,  # Controls randomness; adjust as needed
        )

        # Extract the paraphrased text from the response
        paraphrased_text = response.choices[0].message.content
        return paraphrased_text.strip()  # Return the result without extra whitespace

    except Exception as e:
        print(f"Error: {e}")
        return "I'm sorry I can't answer that question, could you ask anything else?"

def highlight_features(review_text):
    """Summarize the review by highlighting pros and cons of the app based on keywords."""
    try:
        # Create the completion request
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "user",
                    "content": f"Read the following review and provide a summarization of the app's features in two categories: Pros and Cons. Only list each feature as a bullet point. If the feature helps or enhances the experience, list it under 'Pros'. If it causes issues or hinders the experience, list it under 'Cons'. If there is no Cons just show Pro, if there is no Pros just show Cons, Review text: '{review_text}'"
                }
            ],
            max_tokens=100,  # Adjust based on desired output length
            temperature=0.7,  # Controls randomness; adjust as needed
        )

        # Extract the summarized text from the response
        summarized_text = response.choices[0].message.content
        return summarized_text.strip()  # Return the result without extra whitespace

    except Exception as e:
        print(f"Error: {e}")
        return "I'm sorry I can't answer that question, could you ask anything else?"

def sentiment_openai(review_text):
    """Give sentiment analysis for this text."""
    try:
        # Create the completion request
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "user",
                    "content": f"You are an sentiment analysist assistant that give return as POSITIVE/NEGATIVE/NEUTRAL for the review_text. Don't give any word besides that three at all. Review Text: '{review_text}'"
                }
            ],
            max_tokens=100,  # Adjust based on desired output length
            temperature=0.7,  # Controls randomness; adjust as needed
        )

        # Extract the paraphrased text from the response
        paraphrased_text = response.choices[0].message.content
        return paraphrased_text.strip()  # Return the result without extra whitespace

    except Exception as e:
        print(f"Error: {e}")
        return "I'm sorry I can't answer that question, could you ask anything else?"

# Example usage
review_text = "<3"
print("Paraphrased Review:")
print(generate_paraphrase(review_text))
print("\nFeature Highlights:")
print(highlight_features(review_text))
print("Sentiment")
print(sentiment_openai(review_text))


Paraphrased Review:
Our user expressed their affection with a heart symbol.

Feature Highlights:
**Pros:**
- Positive user sentiment indicated by the heart symbol. 

**Cons:**
- None.
Sentiment
POSITIVE


In [ ]:
# Collecting train_df
data_samples = train_df
df = df_new

In [ ]:
import json
from collections import Counter

# List to hold JSON entries
jsonl_data = []


def compare_versions(version1, version2):
    """Compares two version strings formatted as a.b.c.d."""
    if not version1 or not version2 or version1 == 'NaN' or version2 == 'NaN':
        return None  # One of the versions is NaN or empty

    # Split the version strings into parts
    parts1 = list(map(int, version1.split('.')))
    parts2 = list(map(int, version2.split('.')))

    # Compare each part in order
    for p1, p2 in zip(parts1, parts2):
        if p1 > p2:
            return 1  # version1 is newer
        elif p1 < p2:
            return -1  # version2 is newer
    return 0  # versions are the same

def get_latest_and_oldest_version(author_app_versions):
    latest_version = None
    oldest_version = None

    for version in author_app_versions:
        if version != 'NaN':  # Ignore 'NaN' values
            # Update latest version
            if latest_version is None or compare_versions(version, latest_version) == 1:
                latest_version = version

            # Update oldest version
            if oldest_version is None or compare_versions(version, oldest_version) == -1:
                oldest_version = version

    return latest_version, oldest_version

def calculate_statistics(data_samples):
    ratings = []
    review_likes = []
    author_likes = Counter()

    for data in data_samples.itertuples(index=False):
        rating = pd.to_numeric(data.review_rating, errors='coerce') if data.review_rating != 'NaN' else None
        version = data.author_app_version if data.author_app_version != 'NaN' else None
        review_like_count = pd.to_numeric(data.review_likes, errors='coerce') if data.review_likes != 'NaN' else None
        author_name = data.author_name if data.author_name != 'NaN' else None

        if rating is not None:
            ratings.append(rating)
        if review_like_count is not None:
            review_likes.append(review_like_count)
        if author_name:
            author_likes[author_name] += review_like_count if review_like_count is not None else 0


    average_rating = sum(ratings) / len(ratings) if ratings else 0
    rating_counts = Counter(ratings)

    average_review_likes = round((sum(review_likes) / len(review_likes) if review_likes else 0),2)
    most_liked_author = author_likes.most_common(1)
    most_liked_author_name = most_liked_author[0][0] if most_liked_author else None
    most_liked_author_value = most_liked_author[0][1] if most_liked_author else 0
    highest_review_likes = max(review_likes) if review_likes else 0
    # Determine lowest rating(s)
    min_count = min(rating_counts.values()) if rating_counts else 0
    lowest_rating_values = [rating for rating, count in rating_counts.items() if count == min_count]

    # Determine highest rating(s)
    max_count = max(rating_counts.values()) if rating_counts else 0
    highest_rating_values = [rating for rating, count in rating_counts.items() if count == max_count]

    return average_rating, rating_counts, lowest_rating_values, highest_rating_values, min_count, max_count, average_review_likes, most_liked_author_name, most_liked_author_value, highest_review_likes

def format_ratings(rating_values, rating_type):
    if not rating_values:
        return f"There are no {rating_type} ratings available."

    if len(rating_values) == 1:
        return f"The {rating_type} rating is {rating_values[0]} stars."
    elif len(rating_values) == 2:
        return f"The {rating_type} ratings are {rating_values[0]} and {rating_values[1]} stars."
    else:
        # For more than two ratings, join them with commas and add "and" before the last one
        all_but_last = ', '.join(map(str, rating_values[:-1]))
        last_rating = rating_values[-1]
        return f"The {rating_type} ratings are {all_but_last}, and {last_rating} stars."

# Calculate statistics
average_rating, rating_counts, lowest_rating_values, highest_rating_values, min_count, max_count, average_review_likes, most_liked_author_name, most_liked_author_value, highest_review_likes = calculate_statistics(df)
author_app_version = df.author_app_version
latest_version, oldest_version = get_latest_and_oldest_version(author_app_version)
lowest_rating_output = format_ratings(lowest_rating_values, "lowest")
highest_rating_output = format_ratings(highest_rating_values, "highest")
avgrating = round((average_rating),2)
totaldf = len(df)




In [2]:
# Define default prompts and completions outside the loop
jsonl_data = []

default_prompts = [
    (f"What is the average rating of the app based on user reviews?", f"The average rating is {avgrating} stars."),
    (f"How many reviews of this app?", f"As of now, there have been a total of {totaldf} reviews submitted for this app."),
    (f"What is the latest version?", f"The latest Spotify app version that users's review is {latest_version}"),
    (f"What is the oldest version?", f"The oldest Spotify app version that users's review is {oldest_version}"),
    (f"What is the lowest rating?", f"{lowest_rating_output} with {min_count} reviews"),
    (f"What is the highest rating?", f"{highest_rating_output} with {max_count} reviews"),
    (f"What is the average review likes?", f"The average review likes is {average_review_likes} likes"),
    (f"What is the most liked author?", f"The most liked author is {most_liked_author_name} with {most_liked_author_value} likes"),
    (f"What is the highest review likes?", f"The highest review likes is {most_liked_author_value} likes")
]

print("DEFAULT PROMPTS")
for prompt, completion in default_prompts:

    json_structure = {
        "messages": [
            {"role": "system", "content":
                "You are an AI spotify app reviewer that analyzed known user review,"
                f"'total_reviews': {totaldf}, 'lowest_rating': {lowest_rating_values} star with {min_count} reviews, "
                f"'highest_rating': {highest_rating_values} star with {max_count} reviews, "
                f"'average_rating': {avgrating}, 'average_review_likes': {average_review_likes}, "
                f"'most_liked_author': {most_liked_author_name}, 'most_liked_author_values': {most_liked_author_value} likes, "
                f"'latest_version': {latest_version}, 'oldest_version': {oldest_version}"
             },
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion}
        ]
    }
    jsonl_data.append(json_structure)
print("")

# Process each entry in the dataset
idx = 0
# Main loop over data_samples for review-specific prompts and completions
print("LOOP PROMPTS")
for index, data in data_samples.iterrows():  # Iterate through DataFrame rows
    idx += 1
    print(f"Process {idx}")
    review_id = data["review_id"]
    author_name = data["author_name"]
    review_text = data["review_text"]
    review_rating = data["review_rating"]
    review_likes = data["review_likes"]
    review_timestamp = data["review_timestamp"]
    pseudo_author = data["pseudo_author_id"]
    version = data["author_app_version"]
    sentiment = sentiment_openai(review_text)

    # Split review_timestamp into date and time
    review_date, review_time = review_timestamp.split(' ')
    paraphrased_text = generate_paraphrase(review_text)

    # Specific prompts and completions for each review
    specific_prompts = [
        # (f"What do users say about this app? (Review by {author_name})", paraphrased_text),
        (f"How would a user rate the app based on their experience on {review_date}?", f"A user rated this app with a score of {review_rating} stars."),
        (f"What do {author_name} say about this app?", paraphrased_text),
        (f"When did {author_name} giving reviews?", f"The user giving reviews on {review_date}")
        # (f"Summarize review from {author_name}", featureshighlight)
    ]


    # Append specific review-based prompts to jsonl_data
    for prompt, completion in specific_prompts:
      json_structure = {
          "messages": [
              {"role": "system", "content": "You are an AI spotify app reviewer that analyzed known user review,"
                                        f"'review_id' : {review_id}, 'author_name' : {author_name}, 'review_date' : {review_date},"
                                        f"'review_time': {review_time}, 'review_likes': {review_likes},"
                                        f"'author_app_version':{version}, 'sentiment_analysis_ofreviewtext':{sentiment},"
                                        f"'review_text': {review_text}"
              },
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": completion}
          ]
      }

      jsonl_data.append(json_structure)


train_l_json = []

for entry in jsonl_data:
    train_l_json.append(entry)
# Save to JSONL
print(train_l_json)
output_path = "./dataset/train-json-file.jsonl"
with open(output_path, "w") as file:
    for entry in train_l_json:
        json.dump(entry, file)  # Write each entry as a separate JSON object
        file.write("\n")  # Add a newline to separate each JSON object


In [ ]:
# with open(output_path, "r") as file:
#     for line in file:
#         print(line.strip())  # Print each line without additional newline characters


{"messages": [{"role": "system", "content": "You are an AI Spotify app reviewer that analyzed known user reviews., 'total_reviews': 3000, 'lowest_rating': [2] star with 123 reviews, 'highest_rating': [5] star with 1941 reviews, 'average_rating': 4.04, 'average_review_likes': 1.34, 'most_liked_author': A Google user, 'most_liked_author_values': 1290 likes, 'latest_version': 8.8.82.634, 'oldest_version': 1.1.0.112"}, {"role": "user", "content": "What is the average rating of the app based on user reviews?"}, {"role": "assistant", "content": "The average rating is 4.04 stars."}]}
{"messages": [{"role": "system", "content": "You are an AI Spotify app reviewer that analyzed known user reviews., 'total_reviews': 3000, 'lowest_rating': [2] star with 123 reviews, 'highest_rating': [5] star with 1941 reviews, 'average_rating': 4.04, 'average_review_likes': 1.34, 'most_liked_author': A Google user, 'most_liked_author_values': 1290 likes, 'latest_version': 8.8.82.634, 'oldest_version': 1.1.0.112"

In [ ]:
modified_json_data = []
with open(output_path, "r") as file:
    for line in file:
        entry = json.loads(line.strip())


        system_content = entry["messages"][0]["content"]
        prompt = entry["messages"][1]["content"]
        completion = entry["messages"][2]["content"]


        general_info = "You are an AI spotify app reviewer that analyzed known user review"
        metadata = system_content.replace(general_info + ",", "").strip()


        modified_entry = {
            "messages": [
                {"role": "system", "content": general_info},
                {"role": "user", "content": f"{prompt}, {metadata}"},
                {"role": "assistant", "content": completion}
            ]
        }

        # Append new metadata json
        modified_json_data.append(modified_entry)

with open("./dataset/train-json-file-fix.jsonl", "w") as file:
    for entry in modified_json_data:
        json.dump(entry, file)
        file.write("\n")  # new line

In [ ]:
data_samples = valid_df
df = df_new

In [3]:
# Define default prompts and completions outside the loop
jsonl_data = []

default_prompts = [
    (f"What is the average rating of the app based on user reviews?", f"The average rating is {avgrating} stars."),
    (f"How many reviews of this app?", f"As of now, there have been a total of {totaldf} reviews submitted for this app."),
    (f"What is the latest version?", f"The latest Spotify app version that users's review is {latest_version}"),
    (f"What is the oldest version?", f"The oldest Spotify app version that users's review is {oldest_version}"),
    (f"What is the lowest rating?", f"{lowest_rating_output} with {min_count} reviews"),
    (f"What is the highest rating?", f"{highest_rating_output} with {max_count} reviews"),
    (f"What is the average review likes?", f"The average review likes is {average_review_likes} likes"),
    (f"What is the most liked author?", f"The most liked author is {most_liked_author_name} with {most_liked_author_value} likes"),
    (f"What is the highest review likes?", f"The highest review likes is {most_liked_author_value} likes")
]

print("DEFAULT PROMPTS")
for prompt, completion in default_prompts:

    json_structure = {
        "messages": [
            {"role": "system", "content":
                "You are an AI spotify app reviewer that analyzed known user review,"
                f"'total_reviews': {totaldf}, 'lowest_rating': {lowest_rating_values} star with {min_count} reviews, "
                f"'highest_rating': {highest_rating_values} star with {max_count} reviews, "
                f"'average_rating': {avgrating}, 'average_review_likes': {average_review_likes}, "
                f"'most_liked_author': {most_liked_author_name}, 'most_liked_author_values': {most_liked_author_value} likes, "
                f"'latest_version': {latest_version}, 'oldest_version': {oldest_version}"
             },
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion}
        ]
    }
    jsonl_data.append(json_structure)
print("")

# Process each entry in the dataset
idx = 0
# Main loop over data_samples for review-specific prompts and completions
print("LOOP PROMPTS")
for index, data in data_samples.iterrows():  # Iterate through DataFrame rows
    idx += 1
    print(f"Process {idx}")
    review_id = data["review_id"]
    author_name = data["author_name"]
    review_text = data["review_text"]
    review_rating = data["review_rating"]
    review_likes = data["review_likes"]
    review_timestamp = data["review_timestamp"]
    pseudo_author = data["pseudo_author_id"]
    version = data["author_app_version"]
    sentiment = sentiment_openai(review_text)

    # Split review_timestamp into date and time
    review_date, review_time = review_timestamp.split(' ')
    paraphrased_text = generate_paraphrase(review_text)
    featureshighlight = highlight_features(review_text)

    # Specific prompts and completions for each review
    specific_prompts = [
        (f"What do users say about this app? (Review by {author_name})", paraphrased_text),
        (f"How would a user rate the app based on their experience on {review_date}?", f"A user rated this app with a score of {review_rating} stars."),
        (f"What do {author_name} say about this app?", paraphrased_text),
        (f"When did {author_name} giving reviews?", f"The user giving reviews on {review_date}")
        # (f"Summarize review from {author_name}", featureshighlight)
    ]


    # Append specific review-based prompts to jsonl_data
    for prompt, completion in specific_prompts:
      json_structure = {
          "messages": [
              {"role": "system", "content": "You are an AI spotify app reviewer that analyzed known user review,"
                                        f"'review_id' : {review_id}, 'author_name' : {author_name}, 'review_date' : {review_date},"
                                        f"'review_time': {review_time}, 'review_likes': {review_likes},"
                                        f"'author_app_version':{version}, 'sentiment_analysis_ofreviewtext':{sentiment},"
                                        f"'review_text': {review_text}"
              },
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": completion}
          ]
      }

      jsonl_data.append(json_structure)


valid_l_json = []

for entry in jsonl_data:
    valid_l_json.append(entry)
# Save to JSONL
print(train_l_json)
output_path = "./dataset/valid-json-file.jsonl"
with open(output_path, "w") as file:
    for entry in valid_l_json:
        json.dump(entry, file)  # Write each entry as a separate JSON object
        file.write("\n")  # Add a newline to separate each JSON object


In [ ]:

# with open(output_path, "r") as file:
#     for line in file:
#         print(line.strip())  # Print each line without additional newline characters


In [ ]:
modified_json_data = []
with open(output_path, "r") as file:
    for line in file:
        entry = json.loads(line.strip())


        system_content = entry["messages"][0]["content"]
        prompt = entry["messages"][1]["content"]
        completion = entry["messages"][2]["content"]


        general_info = "You are an AI spotify app reviewer that analyzed known user review"
        metadata = system_content.replace(general_info + ",", "").strip()


        modified_entry = {
            "messages": [
                {"role": "system", "content": general_info},
                {"role": "user", "content": f"{prompt}, {metadata}"},
                {"role": "assistant", "content": completion}
            ]
        }

        # Append new metadata json
        modified_json_data.append(modified_entry)

with open("./dataset/valid-json-file-fix.jsonl", "w") as file:
    for entry in modified_json_data:
        json.dump(entry, file)
        file.write("\n")  # new line

In [ ]:
train_file_path = './dataset/train-json-file-fix.jsonl'
valid_file_path = './dataset/valid-json-file-fix.jsonl'

with open(train_file_path, "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

with open(valid_file_path, "rb") as f:
    valid_file = openai.files.create(file=f, purpose="fine-tune")

In [ ]:
# This part is not necessary, only to connect the visualization during fine-tuning in wandb dashboard
# wandb_integration = {"type": "wandb", "wandb": {"project": "gpt-pricer"}}

modelFINETUNING = 'gpt-4o-mini-2024-07-18'
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=valid_file.id,
    model=modelFINETUNING,
    seed=42,
    hyperparameters={"n_epochs": 5, "batch_size": 8}, # using gpu (cuda)
    # integrations = [wandb_integration],
    suffix="spotify-chatbot-reviewer"
)

In [ ]:
#result model
ftmodel =  "ft:gpt-4o-mini-2024-07-18:personal:spotify-chatbot-reviewer-updated:AO57Uu5E"